In [239]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sqlite3

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

# Setting engine for SQLite connection
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)


## Starting SQLite Connection

In [240]:
# Checking classes were made successfully: https://stackoverflow.com/questions/42946174/sqlalchemy-automap-not-generating-base-classes-table-name
engine = create_engine("sqlite:///birthdata.sqlite", echo=False)

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
print(Base.classes.keys())

# Create a session
session = Session(engine)


['clinic2010_county', 'clinic2010_state', 'clinic2015_county', 'clinic2015_state', 'county', 'national', 'outcomes']


### Naming Classes

In [241]:
# Assign the classes to variables
County2015 = Base.classes.clinic2015_county
State2015 = Base.classes.clinic2015_state
County2010 = Base.classes.clinic2010_county
State2010 = Base.classes.clinic2010_state
Outcomes = Base.classes.outcomes
County_births = Base.classes.county
State_births = Base.classes.national

## 1 Table Approach

### Selecting 1 Table for Modeling

In [242]:
# Switch out "selected" with the table of interest 
selected = State2010

# Display the row's columns and data in dictionary format
first_row = session.query(selected).first()
first_row.__dict__


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd20a99f220>,
 'fed_client': 1054270,
 'total_title10': 4111,
 'pp_client_tt': 1769760,
 'index': 0,
 'dept_clinic_client': 1834800,
 'hospital_client_tt': 221220,
 'fed_center_tt': 557,
 'other_clinic_client_tt': 607830,
 'state': 'US Total',
 'hospital_client': 511310,
 'dept_clinic_tt': 2180,
 'total_clinics': 8409,
 'pp_client': 2407840,
 'hospital_tt': 192,
 'fed_center': 3165,
 'other_clinic_client': 898070,
 'pp_tt': 553,
 'dept_clinic': 2439,
 'total_client_tt': 4724250,
 'other_clinic_tt': 629,
 'hospital': 664,
 'fed_client_tt': 435570,
 'total_client': 6706280,
 'pp': 817,
 'dept_clinic_client_tt': 1689860,
 'other_clinic': 1324}

In [134]:
first_row = session.query(County_births).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd2d5391dc0>,
 'combined_fips_code': '1001',
 'state_fips_code': '01',
 'state': 'Alabama',
 'index': 0,
 'upper_confidence_limit': 55.2,
 'birth_rate': 48.4,
 'county_fips_code': '1',
 'county': 'Autauga',
 'year': 2003,
 'lower_confidence_limit': 42.1}

### Getting Query Results from 1 Table

In [153]:
# https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html#par_textimage_1574439295

import csv


In [106]:
url = "state_populations.csv"
state_populations = pd.read_csv(url)

In [107]:
state_populations.head()

,state,2010_population,2015_population
0,United States,309321666,320635163
1,Alabama,4785437,4852347
2,Alaska,713910,737498
3,Arizona,6407172,6829676
4,Arkansas,2921964,2978048


In [108]:
state_populations.dtypes

state              object
2010_population     int64
2015_population     int64
dtype: object

In [348]:
state_expenditures = pd.read_csv("family_planning_expenditures.csv", header=0, names=["state", "MCH block grant", "Medicaid_spending", "SSBG and TANF", "Title10_spending", "state appropriations", "total_spending"])

In [349]:
state_expenditures.dtypes

state                   object
MCH block grant         object
Medicaid_spending        int64
SSBG and TANF           object
Title10_spending         int64
state appropriations    object
total_spending           int64
dtype: object

In [350]:
state_expenditures = state_expenditures[["state","Medicaid_spending", "Title10_spending","total_spending"]]


In [351]:
state_expenditures.head()

,state,Medicaid_spending,Title10_spending,total_spending
0,Alabama,58362,58362,69742
1,Alaska,2812,2812,10158
2,Arizona,52006,52006,57560
3,Arkansas,6899,6899,11397
4,California,438559,438559,454706


In [155]:
county_populations = pd.read_csv("county_populations.csv", encoding='latin-1')

In [156]:
county_populations.head(10)

,state,county,2010_population,2015_population
0,Iowa,Adair,7682,7145
1,Kentucky,Adair,18656,19162
2,Missouri,Adair,25607,25353
3,Oklahoma,Adair,22683,22259
4,Colorado,Adams,441603,490443
5,Idaho,Adams,3976,3891
6,Illinois,Adams,67103,66835
7,Indiana,Adams,34387,34945
8,Iowa,Adams,4029,3754
9,Mississippi,Adams,32297,31561


In [157]:
county_populations.dtypes

state              object
county             object
2010_population     int64
2015_population     int64
dtype: object

## Multi-Table Approach

### Getting Query Results from Several Tables

In [314]:
#states



# This cell can be used if you want to compare data from different tables.
# 1) Identify tables before the period and consider what is the most logical way to merge them.
# 2) Pull in the columns of interest from each table 
# 3) Write out their join, keeping in mind 1 to 1, many to 1, etc. 
# 4) Update column names to match, keeping the first column as the element they were joined on. 
# 5) Update index to that joined column, renaming as needed 
# Resource: https://www.kite.com/python/answers/how-to-join-multiple-tables-together-in-sqlalchemy-in-python#:~:text=Use%20Query.,sequence%20to%20tables%20to%20join.


join_query = session.query(State2010.total_clinics, State2010.total_title10, State2010.pp,\
                           State2010.dept_clinic, State2010.dept_clinic_tt, State2010.hospital_tt,\
                           State2010.hospital, State2010.pp_tt, State2010.pp_client, \
                           State2010.total_client,State2010.total_client_tt,\
                           State_births.state_rate, State_births.year, State_births.state)\
                    .join(State_births, State_births.state == State2010.state)\
                    .filter(State_births.year=="2011").filter(State_births.age_group=="15-19 years")


state_df = pd.DataFrame(join_query, columns=["total_clinics", "total_title10", "total_pp", "dept_clinic", "dept_clinic_tt", "hospital_tt", "hospital", "pp_tt", "pp_client", "total_client", "total_client_tt", "birth_rate", "year", "state"])
#joined_df.set_index('state_name', inplace=True, )

# See code above to change data types if needed
# Drop NaN rows, which will mess with the ML
state_df = state_df.dropna()



In [205]:
# counties

join_query = session.query(County2010.total_clinics, County2010.total_title10, County2010.pp,\
                           County2010.dept_clinic, County2010.hospital, County2010.total_client_tt,
                           County2010.pp_client, County2010.dept_clinic_tt, County_births.birth_rate,\
                           County_births.year, County_births.state, County_births.county)\
                    .join(County_births, County_births.combined_fips_code == County2010.fips)\
                    .filter(County_births.year=="2010")


county_df = pd.DataFrame(join_query, columns=["total_clinics", "total_title10", "total_pp", "health_dept_clinics", "hospitals","title_10_clients","pp_clients", "dept_clinic_title10","birth_rate", "year", "state", "county"])
#joined_df.set_index('state_name', inplace=True, )

# See code above to change data types if needed
# Drop NaN rows, which will mess with the ML
county_df = county_df.dropna()

In [206]:
county_df

,total_clinics,total_title10,total_pp,health_dept_clinics,hospitals,title_10_clients,pp_clients,dept_clinic_title10,birth_rate,year,state,county
0,2,1,0,1,0,1120.0,0,1,34.8,2010,Alabama,Autauga
1,3,1,0,1,0,1250.0,0,1,41.2,2010,Alabama,Baldwin
2,3,2,0,2,0,1050.0,0,2,64.8,2010,Alabama,Barbour
3,2,1,0,1,0,670.0,0,1,49.7,2010,Alabama,Bibb
4,2,1,0,1,0,1520.0,0,1,45.7,2010,Alabama,Blount
...,...,...,...,...,...,...,...,...,...,...,...,...
3130,1,1,0,0,0,1160.0,0,0,49.6,2010,Wyoming,Sweetwater
3131,1,1,0,0,0,410.0,0,0,23.3,2010,Wyoming,Teton
3132,2,2,0,0,0,690.0,0,0,40.0,2010,Wyoming,Uinta
3133,2,1,0,1,0,360.0,0,1,36.5,2010,Wyoming,Washakie


In [207]:
county_df["year"].unique()

array([2010])

In [208]:
county_df = county_df.merge(county_populations, how='left', on=["state","county"])

In [209]:
county_df = county_df.dropna()
county_df.shape

(3080, 14)

In [210]:
county_df.columns

Index(['total_clinics', 'total_title10', 'total_pp', 'health_dept_clinics',
       'hospitals', 'title_10_clients', 'pp_clients', 'dept_clinic_title10',
       'birth_rate', 'year', 'state', 'county', '2010_population',
       '2015_population'],
      dtype='object')

In [211]:
county_df["clinics_per_capita"] = county_df["total_clinics"]/county_df["2010_population"]
county_df["title10_per_capita"] = county_df["total_title10"]/county_df["2010_population"]
county_df["pp_per_capita"] = county_df["total_pp"]/county_df["2010_population"]
county_df["health_dept_per_capita"] = county_df['health_dept_clinics']/county_df["2010_population"]
county_df["hospitals_per_capita"] = county_df['hospitals']/county_df["2010_population"]
county_df["title_10_clients_per_capita"] = county_df["title_10_clients"]/county_df["2010_population"]
county_df["pp_clients_per_capita"] = county_df['pp_clients']/county_df["2010_population"]
county_df["dept_clinic_title10_per_capita"] = county_df['dept_clinic_title10']/county_df["2010_population"]


In [212]:
county_df.head()

,total_clinics,total_title10,total_pp,health_dept_clinics,hospitals,title_10_clients,pp_clients,dept_clinic_title10,birth_rate,year,...,2010_population,2015_population,clinics_per_capita,title10_per_capita,pp_per_capita,health_dept_per_capita,hospitals_per_capita,title_10_clients_per_capita,pp_clients_per_capita,dept_clinic_title10_per_capita
0,2,1,0,1,0,1120.0,0,1,34.8,2010,...,54571.0,54864.0,0.000037,0.000018,0.0,0.000018,0.0,0.020524,0.0,0.000018
1,3,1,0,1,0,1250.0,0,1,41.2,2010,...,182265.0,202939.0,0.000016,0.000005,0.0,0.000005,0.0,0.006858,0.0,0.000005
2,3,2,0,2,0,1050.0,0,2,64.8,2010,...,27457.0,26283.0,0.000109,0.000073,0.0,0.000073,0.0,0.038242,0.0,0.000073
3,2,1,0,1,0,670.0,0,1,49.7,2010,...,22915.0,22566.0,0.000087,0.000044,0.0,0.000044,0.0,0.029238,0.0,0.000044
4,2,1,0,1,0,1520.0,0,1,45.7,2010,...,57322.0,57526.0,0.000035,0.000017,0.0,0.000017,0.0,0.026517,0.0,0.000017


In [315]:
state_df = state_df.merge(state_populations, on="state", how="inner")

In [353]:
state_df = state_df.merge(state_expenditures, on="state", how="inner")

In [346]:
state_df.head()

,total_clinics,total_title10,total_pp,dept_clinic,dept_clinic_tt,hospital_tt,hospital,pp_tt,pp_client,total_client,...,pp_per_capita,health_dept_per_capita,hospitals_per_capita,title_10_clients_per_capita,pp_clients_per_capita,dept_clinic_title10_per_capita,percent_clinics_tt_funded,percent_depts_tt_funded,percent_pp_tt_funded,percent_clients_at_pp
0,168,80,2,76,75,0,2,0,280,115460,...,4.179347e-07,0.000016,4.179347e-07,0.021662,0.000059,0.000016,0.476190,0.986842,0.000000,0.002425
1,118,6,5,15,2,0,1,3,5460,23500,...,7.003684e-06,0.000021,1.400737e-06,0.009539,0.007648,0.000003,0.050847,0.133333,0.600000,0.232340
2,184,37,18,34,20,2,15,9,36390,97610,...,2.809352e-06,0.000005,2.341126e-06,0.006671,0.005680,0.000003,0.201087,0.588235,0.500000,0.372810
3,145,89,2,89,89,0,0,0,4270,83940,...,6.844711e-07,0.000030,0.000000e+00,0.026376,0.001461,0.000030,0.613793,1.000000,0.000000,0.050870
4,1085,325,102,140,56,9,173,95,686680,1529820,...,2.733155e-06,0.000004,4.635646e-06,0.029496,0.018400,0.000002,0.299539,0.400000,0.931373,0.448863


In [317]:
state_df["clinics_per_capita"] = state_df["total_clinics"]/state_df["2010_population"]
state_df["title10_per_capita"] = state_df["total_title10"]/state_df["2010_population"]
state_df["pp_per_capita"] = state_df["total_pp"]/state_df["2010_population"]
state_df["health_dept_per_capita"] = state_df['dept_clinic']/state_df["2010_population"]
state_df["hospitals_per_capita"] = state_df['hospital']/state_df["2010_population"]
state_df["title_10_clients_per_capita"] = state_df["total_client_tt"]/state_df["2010_population"]
state_df["pp_clients_per_capita"] = state_df['pp_client']/state_df["2010_population"]
state_df["dept_clinic_title10_per_capita"] = state_df['dept_clinic_tt']/state_df["2010_population"]
state_df["percent_clinics_tt_funded"] = state_df['total_title10']/state_df["total_clinics"]
state_df["percent_depts_tt_funded"] = state_df['dept_clinic_tt']/state_df["dept_clinic"]
state_df["percent_pp_tt_funded"] = state_df['pp_tt']/state_df["total_pp"]
state_df["percent_clients_at_pp"] = state_df['pp_client']/state_df["total_client"]


In [324]:
state_df=state_df.dropna()

In [325]:
state_df.columns

Index(['total_clinics', 'total_title10', 'total_pp', 'dept_clinic',
       'dept_clinic_tt', 'hospital_tt', 'hospital', 'pp_tt', 'pp_client',
       'total_client', 'total_client_tt', 'birth_rate', 'year', 'state',
       '2010_population', '2015_population', 'clinics_per_capita',
       'title10_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita',
       'percent_clinics_tt_funded', 'percent_depts_tt_funded',
       'percent_pp_tt_funded', 'percent_clients_at_pp'],
      dtype='object')

## Prepping the Model Data

In [326]:
# Close the session from before
session.close()

# Assign the data to X and y
# Note: Sklearn requires a two-dimensional array of values so we use reshape to create this

X = state_df[['clinics_per_capita',
       'title10_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita',
       'percent_clinics_tt_funded', 'percent_depts_tt_funded',
       'percent_pp_tt_funded', 'percent_clients_at_pp']]
y = state_df["birth_rate"]

feature_names = ['clinics_per_capita',
       'title10_per_capita', 'pp_per_capita', 'health_dept_per_capita',
       'hospitals_per_capita', 'title_10_clients_per_capita',
       'pp_clients_per_capita', 'dept_clinic_title10_per_capita',
       'percent_clinics_tt_funded', 'percent_depts_tt_funded',
       'percent_pp_tt_funded', 'percent_clients_at_pp']

print("Shape: ", X.shape, y.shape)

Shape:  (42, 12) (42,)


In [327]:
# Use sklearn's `train_test_split` to split the data into training and testing
# Keep in mind that our group may want to use the model on an entirely different table as well. 

from sklearn.model_selection import train_test_split # may not apply to all models

### BEGIN SOLUTION 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### END SOLUTION

## Linear Regression

In [328]:
# Create the model

### BEGIN SOLUTION

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(max_depth=4, n_estimators=100, random_state=42)

### END SOLUTION

In [329]:
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, random_state=42)

In [330]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data

from sklearn.metrics import mean_squared_error, r2_score

### BEGIN SOLUTION

# Use our model to make predictions
predicted = model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

### END SOLUTION

Mean Squared Error (MSE): 61.11363854375168
R-squared (R2 ): 0.3161999719078663


In [331]:
# Call the `score` method on the model to show the r2 score

### BEGIN SOLUTION
model.score(X_test, y_test)
### END SOLUTION

0.3161999719078663

In [332]:
sorted(zip(model.feature_importances_, feature_names), reverse=True)

[(0.24916878913294233, 'pp_clients_per_capita'),
 (0.2441331650066089, 'pp_per_capita'),
 (0.11722980886319008, 'percent_clients_at_pp'),
 (0.09198452735537278, 'percent_pp_tt_funded'),
 (0.05773260684670149, 'percent_clinics_tt_funded'),
 (0.053918022708138884, 'clinics_per_capita'),
 (0.04436961142449079, 'health_dept_per_capita'),
 (0.04400722950504308, 'title_10_clients_per_capita'),
 (0.03678582950384224, 'dept_clinic_title10_per_capita'),
 (0.02322775752220244, 'percent_depts_tt_funded'),
 (0.02212184137422273, 'title10_per_capita'),
 (0.015320810757244398, 'hospitals_per_capita')]

## 

In [ ]:
#R2 with raw state data = 0.26
#R2 with per capita = 0.15
#r2 pp, title10, total clinics, states raw plus per capita = .29, pp per capita highest feature importance